In [9]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np
import time

import matplotlib
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [10]:
train = pd.read_csv('C:/Users/1/Python/Kaggle/Elo loyalty card/train_clean.csv', nrows = 1000)
test = pd.read_csv('C:/Users/1/Python/Kaggle/Elo loyalty card/test_clean.csv', nrows = 1000)

target = train['target']

In [12]:
#train['outliers'] = 0
#train.loc[train['target'] < -30, 'outliers'] = 1
#train = train[train['outliers'] == 0]
#target = train['target']

In [13]:
categorical = ['feature_1', 'feature_2', 'feature_3', 'installments', 'city_id', 'category_2']
objects = ['first_active_month','card_id','outliers', 'target']
features = [c for c in train.columns if c not in objects]

In [14]:
param = {'max_depth': 3,
              'num_leaves': 50,
              'min_data_in_leaf': 20,
              "feature_fraction": 0.8,
             "bagging_fraction": 0.8,
              "lambda_l1": 1,
              "bagging_freq": 3,
              "min_child_samples": 20,
              'objective':'regression',
              'learning_rate': 0.1,
              "boosting": "gbdt",
              "bagging_seed": 2018,
              "metric": 'rmse',
              "random_state": 4590}
    
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])#, categorical_feature=categorical)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])#, categorical_feature=categorical)

    num_round = 5000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.08278	valid_1's rmse: 1.7739
[200]	training's rmse: 0.809208	valid_1's rmse: 1.81979
Early stopping, best iteration is:
[20]	training's rmse: 1.41999	valid_1's rmse: 1.70438
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.08883	valid_1's rmse: 1.81861
[200]	training's rmse: 0.807123	valid_1's rmse: 1.87753
Early stopping, best iteration is:
[53]	training's rmse: 1.26787	valid_1's rmse: 1.76587
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.12697	valid_1's rmse: 1.60036
[200]	training's rmse: 0.857356	valid_1's rmse: 1.66322
Early stopping, best iteration is:
[2]	training's rmse: 1.67344	valid_1's rmse: 1.49969
fold n°3
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.09558	valid_1's rmse: 1.79982
[200]	training's rmse: 0.846141	valid_1's rmse: 1.85074
Ear

In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission.csv", index=False)